# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [45]:
dataset_name='pokemon'

In [46]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [47]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')

In [48]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [49]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [50]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [51]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [52]:
import tensorflow as tf

In [53]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [54]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [55]:
_instance_scale

1.0

In [56]:
import numpy as np
from collections.abc import Iterable

In [57]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [58]:
_outputs_shape

120000

# Model's Layers definition

In [59]:
_uf=20
c=50
menc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=_uf*c*c, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, _uf)),
    tf.keras.layers.Conv2DTranspose(filters=_uf, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=_uf*3, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [60]:
model_name = dataset_name+'ConvTVAAE'
recoding_dir='recoding'+sep_local+model_name

In [61]:
from training.adversarial.autoencoders.VAAE import VAAE as AE

In [62]:
inputs_shape=IMG_SIZE

In [63]:
variables_params = \
[
    {
        'name': 'encoder_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays
    }

    ,
    
        {
        'name': 'encoder_logvar', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [64]:
from utils.data_and_files.file_utils import create_if_not_exist

In [65]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [66]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonConvVAAE\\var_save_dir'

In [32]:
#to restore trained model, set filepath=_restore

In [ ]:
from stats.traditional_adversarial_losses import \
    create_latent_adversarial_real_losses, \
    create_latent_adversarial_fake_losses, \
    create_latent_adversarial_losses

latent_adversarial_losses = {
    'latent_adversarial_losses': create_latent_adversarial_losses,
    'latent_adversarial_real_losses': create_latent_adversarial_real_losses,
    'latent_adversarial_fake_losses': create_latent_adversarial_fake_losses,
}

In [33]:
ae = AE( 
    name=model_name,
    latent_adversarial_losses=latent_adversarial_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "encoder_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_mean_inputs (InputLa [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 99, 99, 20)        560       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 180)       32580     
_________________________________________________________________
flatten (Flatten)            (None, 432180)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                13829792  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)               

  WARNING  | None


Model: "encoder_logvar"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_logvar_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 99, 99, 20)        560       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 49, 49, 180)       32580     
_________________________________________________________________
flatten_1 (Flatten)          (None, 432180)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                13829792  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)             

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 50000)             1650000   
_________________________________________________________________
reshape (Reshape)            (None, 50, 50, 20)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 100, 100, 20)      3620      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 200, 200, 60)      10860     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 200, 200, 3)       1623      
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       1

  WARNING  | None


In [34]:
ae.compile()

Model: "pokemonConvVAAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_logvar_inputs (InputLay [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
encoder_logvar (Model)          (None, 32)           13863060    encoder_logvar_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           encoder_logvar[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
____________________________________________________________________________________

In [35]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [36]:
progbar = NotebookPrograssBar(leave_outer=False)

In [37]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [38]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [39]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonConvVAAE\\csv_dir\\pokemonConvVAAE.csv'

In [40]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [41]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [42]:
import numpy as np

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)